In [17]:
from pathlib import Path
from bs4 import BeautifulSoup,  NavigableString
import ray

In [24]:
ray.data.DataContext.get_current().execution_options.verbose_progress = True

In [22]:
DOCS_DIR = Path('./data', "docs.ray.io/en/master/")
ds = ray.data.from_items([{"path": path} for path in DOCS_DIR.rglob("*.html") if not path.is_dir()])
print(f"{ds.count()} documents")

2932 documents


In [7]:
ds.take(1)

2023-10-11 15:56:37,970	INFO dataset.py:2380 -- Tip: Use `take_batch()` instead of `take() / show()` to return records in pandas or numpy batch format.
2023-10-11 15:56:37,972	INFO streaming_executor.py:93 -- Executing DAG InputDataBuffer[Input] -> LimitOperator[limit=1]
2023-10-11 15:56:37,973	INFO streaming_executor.py:94 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
2023-10-11 15:56:37,974	INFO streaming_executor.py:96 -- Tip: For detailed progress reporting, run `ray.data.DataContext.get_current().execution_options.verbose_progress = True`


[dataset]: Run `pip install tqdm` to enable progress reporting.


[{'path': PosixPath('data/docs.ray.io/en/master/search.html')}]

In [20]:
def extract_text_from_section(section):
    texts = []
    for elem in section.children:
        if isinstance(elem, NavigableString):
            if elem.strip():
                texts.append(elem.strip())
        elif elem.name == "section":
            continue
        else:
            texts.append(elem.get_text().strip())
    return "\n".join(texts)

def path_to_uri(path, scheme="https://", domain="docs.ray.io"):
    return scheme + domain + str(path).split(domain)[-1]
    
def extract_sections(record):
    with open(record["path"], "r", encoding="utf-8") as html_file:
        soup = BeautifulSoup(html_file, "html.parser")
    sections = soup.find_all("section")
    section_list = []
    for section in sections:
        section_id = section.get("id")
        section_text = extract_text_from_section(section)
        if section_id:
            uri = path_to_uri(path=record["path"])
            section_list.append({"source": f"{uri}#{section_id}", "text": section_text})
    return section_list

In [21]:
sample_html_fp = Path('./data', "docs.ray.io/en/master/rllib/rllib-env.html")
extract_sections({"path": sample_html_fp})[0]

{'source': 'https://docs.ray.io/en/master/rllib/rllib-env.html#environments',
 'text': '\nEnvironments#\nRLlib works with several different types of environments, including Farama-Foundation Gymnasium, user-defined, multi-agent, and also batched environments.\nTip\nNot all environments work with all algorithms. Check out the algorithm overview for more information.\n'}

In [25]:
sections_ds = ds.flat_map(extract_sections)
sections_ds.count()

2023-10-11 16:11:22,549	INFO streaming_executor.py:93 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[FlatMap(extract_sections)]
2023-10-11 16:11:22,550	INFO streaming_executor.py:94 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=True)


5489

In [34]:
sections_ds.take(1)[0]['text']

2023-10-11 16:14:07,065	INFO streaming_executor.py:93 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[FlatMap(extract_sections)] -> LimitOperator[limit=1]
2023-10-11 16:14:07,066	INFO streaming_executor.py:94 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=True)


'\nTesting Autoscaling Locally#\nTesting autoscaling behavior is important for autoscaler development and the debugging of applications that depend\non autoscaler behavior. You can run the autoscaler locally without needing to launch a real cluster with one of the\nfollowing methods:'

In [35]:
section_lengths = []
for section in sections_ds.take_all():
    section_lengths.append(len(section["text"].split()))



In [37]:
from preprocessing import FileIO

ModuleNotFoundError: No module named 'loguru'

Bad pipe message: %s [b'\xaa\xa1\x81{\x1e\xe3\xb6\xaa#\xf7\x92"\xe8\x88E\x1a\x06\xd4 \xbf\x16J\xd4\x82\xfdzz\x9a\x80w\x83\xb6\x14\x89\xfc=\x07\xbb\xcfG\x1bP\nJDOZ\xc5\xd5\xe2\xb6\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08', b'\x01\x05\x01\x06\x01']
Bad pipe message: %s [b'\xf3L\xa2\xbf\xc4\xb6\xdaRz\xb2m\xfe){ \x85\xf0\xbd\x00\x00|\xc0,\xc00\x00', b"\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P\x00=\x00<\x005\x00/\x00\x9a\x00\x99\xc0\x07\xc0